In [ ]:
import requests
import pandas as pd
import ast
import time
from bs4 import BeautifulSoup
import google.generativeai as genai

superhero_api_access_token = "671cbe3d444124af8b8669c154d2264f"
superhero_api_base_url = f"https://superheroapi.com/api/{superhero_api_access_token}"

comic_vine_api_keys = [
    "de2052c788a1d8f4b86e98cd90e3a1c7c4b16318",
    "07153cc61a942dd52742ec5653c21bd336455f42",
    "754e7fa3bde70bad75bbded03a3ba6de2e3a6760",
    "9ff99d4ddd3f499b8025fa640b20d486aefa0e6c",
    "e07ed65b468225d9f42e62d873946f96f242ec4d",
    "e97cc1c2a58eaf0663299531e9af3635f3e3640e",
    "ea15139fd1138436f785c459e94352cd1afca238",
    "44abbc08d124331e9e78b532c22c8e344e06b28a",
    "730b74ee90c877436c2c3358616b94801fef46f0",
    "85b7ddc8b83900935d67c6573cf933a0ba0a85de",
    "e8d51f9ea128b3535f5ad9f66dd8f59f9099c5e2"
    ]

comic_vine_base_url = f"https://comicvine.gamespot.com/api/"


comic_vine_api_keys_index = 0
total_request_count = 0
request_count = 0

In [53]:
# --------------------
# Other functions
# --------------------

# Convert a value to float if possible, to None if not        
def convert_to_float_or_none(value):
    try:
        return float(value)
    except ValueError:
        return None

# Convert a string to list using either ";", "\n" or "," for separation    
def set_string_to_list(input_string, except_string = ""):
    if input_string == except_string or input_string == "null" or input_string == "" or input_string == "-" or input_string == None:
        return []
    elif ";" in input_string:
        return [phrase.strip() for phrase in input_string.split(";")]
    elif "\n" in input_string:
        return [phrase.strip() for phrase in input_string.split("\n")]
    else:
        return [phrase.strip() for phrase in input_string.split(",")]

# Check if the string is None or equivalent  
def process_string(input_string, except_string = ""):
    if input_string == except_string or input_string == "null" or input_string == "" or input_string == "-" or input_string == None or input_string == "None":
        return None
    else:
        return input_string

# Detect if the string contain "[" then convert the string to list
def try_literal_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) and val.startswith("[") else val
    except (SyntaxError, ValueError):
        return val

# Delete None and 'None' from list.
def remove_none_values(lst):
    if isinstance(lst, list):
        return [item for item in lst if item is not None and item != "None"]
    return lst

# Extract keys inside include_list from dictionary data.
def extract_from_dict(data, include_list):
        return {key: data.get(key, "") for key in include_list}

#Extract keys inside include_list from dictionarys list dict_list.
def extract_from_dict_list(dict_list, include_list):
        if isinstance(dict_list, list):
            return [extract_from_dict(item, include_list) for item in dict_list]
        return dict_list

#Extract values from the key target_key from dictionarys list dict_list.
def extract_values_by_key(dict_list, target_key):
    if isinstance(dict_list, list):
        new_list = [item.get(target_key, "") for item in dict_list]
        return remove_none_values(new_list)
    return []

#Extract value from key target_key from dictionary le_dict.
def extract_value_by_key(data, target_key):
    if isinstance(data, str):
        try:
            data = ast.literal_eval(data)
        except (ValueError, SyntaxError):
            return data

    if isinstance(data, dict):
        return str(data.get(target_key, ""))
    
    return data

# Delete all <...> from text
def remove_html_tags(text):
    if isinstance(text, str):
        return BeautifulSoup(text, "html.parser").get_text(separator=" ").strip()
    return text

# Define genders
def define_gender(int_gender):
    if int_gender == 1:
        return "Male"
    elif int_gender == 2:
        return "Female"
    else:
        return "Other"

In [ ]:
# --------------------
# Functions used to search ComicVine characters thanks to the API 
# --------------------

# Verify if current API Key as less than 150 request made if yes change it and ask you to change IP
def comic_vine_api_selector():
    global total_request_count, request_count, comic_vine_api_keys_index
    request_count += 1
    if request_count >= 151:
        comic_vine_api_keys_index += 1
        request_count = 0
        if comic_vine_api_keys_index >= len(comic_vine_api_keys):
            comic_vine_api_keys_index = 0
        input("Changez l'IP! Appuyez sur Entrée pour continuer...")

    api_selected = comic_vine_api_keys[comic_vine_api_keys_index]
    return api_selected

# Make comic vine request and manage error and bugs 
def make_cv_request(url,headers):
    for i in range(0,10):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                if 'results' in response.json():
                    return response.json()
                else:
                    print('Comicvine est un fdp')
            else:
                print("Comic vine envoie un error", response.status_code)
        except:
            print('Comivine casse les couilles',i,'fois')

# search comic vine character with name using API
def search_character_by_name_with_comic_vine(name):
    name = name.replace(" ","%20")
    api_key = comic_vine_api_selector()
    url = f"{comic_vine_base_url}characters/?api_key={api_key}&format=json&filter=name:{name}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = make_cv_request(url,headers)
    return response

# search comic vine character with id using API
def search_character_by_id_with_comic_vine(id):
    url = f"{comic_vine_base_url}character/4005-{id}/?api_key={comic_vine_api_selector()}&format=json"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = make_cv_request(url,headers)
    return response

In [5]:
# --------------------
# Functions to create characters ComicVine with all informations (mostly full dict)
# --------------------

def create_comic_vine_character_data(character_result):
    character_data = {
        "ID": character_result.get("id", ""),
        "Aliases": set_string_to_list(character_result.get("aliases", "")),
        "Name": character_result.get("name", ""),
        "Birth": character_result.get("birth", ""),
        "Character_enemies": extract_from_dict_list(character_result.get("character_enemies", ""),["id","name"]),
        "Character_friends": extract_from_dict_list(character_result.get("character_friends", ""),["id","name"]),
        "Count_of_issue_appearances": character_result.get("count_of_issue_appearances", ""),
        "Creators": extract_from_dict_list(character_result.get("creators", ""),["id","name"]),
        "Date_added": character_result.get("date_added", ""),
        "Date_last_updated": character_result.get("date_last_updated", ""),
        "Deck": process_string(character_result.get("deck", "")),
        "Description": process_string(character_result.get("description", "")),
        "First Appearance": extract_from_dict_list(character_result.get("first_appeared_in_issue", ""),["id","name"]),
        "Gender": character_result.get("gender", ""),
        "Image": character_result.get("image", ""),
        "Issue_credits": extract_from_dict_list(character_result.get("issue_credits", ""),["id","name"]),
        "Issues_died_in": extract_from_dict_list(character_result.get("issues_died_in", ""),["id","name"]),
        "Movies": extract_from_dict_list(character_result.get("movies", ""),["id","name"]),
        "Race": extract_from_dict(character_result.get("origin", ""),["id","name"]),
        "Powers": extract_from_dict_list(character_result.get("powers", ""),["id","name"]),
        "Publisher": extract_from_dict(character_result.get("publisher", ""),["id","name"]),
        "Full Name": process_string(character_result.get("real_name", "")),
        "Team_enemies": extract_from_dict_list(character_result.get("team_enemies", ""),["id","name"]),
        "Team_friends": extract_from_dict_list(character_result.get("team_friends", ""),["id","name"]),
        "Teams": extract_from_dict_list(character_result.get("teams", ""),["id","name"]),
        "Volume_credits": extract_from_dict_list(character_result.get("volume_credits", ""),["id","name"]),
    }
    return character_data

def create_comic_vine_charcaters_from_superheroAPI_df(data, csv_name, start_id = 0):
    cv_data = []
    for i, row in data.iterrows():
        id_shapi = row['id_shapi']
        if id_shapi >= start_id:
            id_cv = int(row['id_cv'])
            print(id_shapi,':',id_cv)
            character_result = search_character_by_id_with_comic_vine(id_cv)['results']
            character_data = create_comic_vine_character_data(character_result)
            character_data['id_shapi'] = id_shapi
            cv_data.append(character_data)
            cv_df = pd.DataFrame(cv_data)
            cv_df.to_csv(csv_name, index=False)
            print("--------------")
            time.sleep(1)


In [ ]:
# --------------------
# Functions to create characters ComicVine with informations filtered
# --------------------

def create_comic_vine_character_data(character_result):
    character_data = {
        "ID": character_result.get("id", ""),
        "Aliases": set_string_to_list(character_result.get("aliases", "")),
        "Name": character_result.get("name", ""),
        "Birth": character_result.get("birth", ""),
        "Character_enemies": extract_from_dict_list(character_result.get("character_enemies", ""),["id","name"]),
        "Character_friends": extract_from_dict_list(character_result.get("character_friends", ""),["id","name"]),
        "Count_of_issue_appearances": character_result.get("count_of_issue_appearances", ""),
        "Creators": extract_value_by_key(character_result.get("creators", ""),"name"),
        "Date_added": character_result.get("date_added", ""),
        "Date_last_updated": character_result.get("date_last_updated", ""),
        "Deck": process_string(character_result.get("deck", "")),
        "Description": remove_html_tags(process_string(character_result.get("description", ""))),
        "First Appearance": extract_value_by_key(character_result.get("first_appeared_in_issue", ""),"name"),
        "Gender": define_gender(character_result.get("gender", "")),
        "Image": character_result.get("image", ""),
        "Issue_credits": extract_values_by_key(character_result.get("issue_credits", ""),"name"),
        "Movies": extract_values_by_key(character_result.get("movies", ""),"name"),
        "Race": extract_value_by_key(character_result.get("origin", ""),"name"),
        "Powers": extract_values_by_key(character_result.get("powers", ""),"name"),
        "Publisher": extract_value_by_key(character_result.get("publisher", ""),"name"),
        "Full Name": process_string(character_result.get("real_name", "")),
        "Team_enemies": extract_values_by_key(character_result.get("team_enemies", ""),"name"),
        "Team_friends": extract_values_by_key(character_result.get("team_friends", ""),"name"),
        "Teams": extract_values_by_key(character_result.get("teams", ""),"name"),
        "Volume_credits": extract_values_by_key(character_result.get("volume_credits", ""),"name"),
    }
    return character_data

def create_comic_vine_charcaters_from_superheroAPI_df(data, csv_name, start_id = 0):
    cv_data = []
    for i, row in data.iterrows():
        id_shapi = row['id_shapi']
        if id_shapi >= start_id:
            id_cv = int(row['id_cv'])
            print(id_shapi,':',id_cv)
            character_result = search_character_by_id_with_comic_vine(id_cv)['results']
            character_data = create_comic_vine_character_data(character_result)
            character_data['id_shapi'] = id_shapi
            cv_data.append(character_data)
            cv_df = pd.DataFrame(cv_data)
            cv_df.to_csv(csv_name, index=False)
            print("--------------")
            time.sleep(1)

In [ ]:
# --------------------
# Functions to create characters from SuperheroAPI
# --------------------

def get_hero_by_id_with_superhero_api(id):
    url = f"{superhero_api_base_url}/{id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Something went wrong"}

def create_SuperHeroAPI_character_data(character_result):
    {
        "ID": character_result.get("id", ""),
        "Name": character_result.get("name", ""),
        "Intelligence": convert_to_float_or_none(character_result.get("powerstats", {}).get("intelligence", "")),
        "Strength": convert_to_float_or_none(character_result.get("powerstats", {}).get("strength", "")),
        "Speed": convert_to_float_or_none(character_result.get("powerstats", {}).get("speed", "")),
        "Durability": convert_to_float_or_none(character_result.get("powerstats", {}).get("durability", "")),
        "Power": convert_to_float_or_none(character_result.get("powerstats", {}).get("power", "")),
        "Combat": convert_to_float_or_none(character_result.get("powerstats", {}).get("combat", "")),
        "Full Name": set_string_to_list(character_result.get("biography", {}).get("full-name", "").replace(", Jr", " Jr"), "No full name found."),
        "Alter Egos": set_string_to_list(character_result.get("biography", {}).get("alter-egos", ""), "No alter egos found."),
        "Aliases": character_result.get("biography", {}).get("aliases", ""),
        "Place Of Birth": process_string(character_result.get("biography", {}).get("place-of-birth", ""), "No place of birth found."),
        "First Appearance": process_string(character_result.get("biography", {}).get("first-appearance", ""), "First appearance unknown"),
        "Publisher": process_string(character_result.get("biography", {}).get("publisher", ""), "No publisher found."),
        "Alignment": process_string(character_result.get("biography", {}).get("alignment", ""), "No alignment found."),
        "Gender": process_string(character_result.get("appearance", {}).get("gender", ""), "No gender found."),
        "Race": process_string(character_result.get("appearance", {}).get("race", ""), "No race found."),
        "Height (cm)": convert_to_float_or_none(character_result.get("appearance", {}).get("height", "")[1].replace(" cm", "")),
        "Weight (kg)": convert_to_float_or_none(character_result.get("appearance", {}).get("weight", "")[1].replace(" kg", "")),
        "Eye Color": process_string(character_result.get("appearance", {}).get("eye-color", ""), "No eye color found."),
        "Hair Color": process_string(character_result.get("appearance", {}).get("hair-color", ""), "No hair color found."),
        "Work": set_string_to_list(character_result.get("work", {}).get("occupation", ""), "No occupation found."),
        "Base": set_string_to_list(character_result.get("work", {}).get("base", ""), "No base found."),
        "Group Affiliation": set_string_to_list(character_result.get("connections", {}).get("group-affiliation", ""), "No group affiliation found."),
        "Relatives": set_string_to_list(character_result.get("connections", {}).get("relatives", ""), "No relatives found."),
        "Image": set_string_to_list(character_result.get("image", {}).get("url", ""), "No image found.")
    }

def create_SuperHeroAPI_characters():
    heroes_data = []
    cpt=0
    for hero_id in range(1,732):
        cpt +=1
        print(cpt)
        hero = get_hero_by_id_with_superhero_api(hero_id)
        if "error" not in hero and hero_id != 198:
            heroes_data.append(create_SuperHeroAPI_character_data(hero)['Results'])
    heroes_df = pd.DataFrame(heroes_data)
    heroes_df_cleaned = heroes_df.map(lambda x: [i for i in x if i != '-'] if isinstance(x, list) else x)
    return heroes_df_cleaned

In [4]:
# --------------------
# Functions to find the comic vine id of SuperHeroAPI characters 
# --------------------

def search_comic_vine_ids(superhero_api_dataframe,min_id = 1):
    dico_id = {
        'id_shapi': [],
        'name' : [],
        'id_cv' : []
    }
    for i, row in superhero_api_dataframe.iterrows():
        id_shapi = row['ID']
        if id_shapi >= min_id:
            id_name = row['Name']
            print("id =", id_shapi, "name =",id_name)
            dico_id['id_shapi'].append(id_shapi)
            dico_id['name'].append(id_name)
            name_list = []
            name_list.append(row['Name'])
            name_list.extend(row['Full Name'])
            name_list.extend(row['Aliases'])
            cv_search = {}
            for name in name_list:
                cv_search = search_character_by_name_with_comic_vine(name)
                
                if len(cv_search['results']) > 0:
                    break
                time.sleep(1)
            if len(cv_search['results']) == 0:
                dico_id['id_cv'].append(None)
            elif len(cv_search['results']) == 1:
                dico_id['id_cv'].append(cv_search['results'][0]['id'])
            else:
                best_cv_index = 0
                best_cv_count_issues = 0
                cpt = 0
                for result in cv_search['results']:
                    cv_count_issues = result['count_of_issue_appearances']
                    if best_cv_count_issues < cv_count_issues:
                        best_cv_index = cpt
                        best_cv_count_issues = cv_count_issues
                    cpt += 1
                dico_id['id_cv'].append(cv_search['results'][best_cv_index]['id'])
            tmp_df_id = pd.DataFrame(dico_id)
            tmp_df_id.to_csv("df_ids2.csv", index=False)
            print("--------------")
            time.sleep(1)

                
            



In [ ]:
# --------------------
# Functions to generate charcter's stats with gemini
# --------------------



def generate_stats_with_gemini(data):
    GOOGLE_API_KEY = "Enter API_Key"
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-2.0-flash')

    system_prompt = """
    Je vais te passer des donnée sur des personnage fictif je veux que tu me renvoie leurs caracteristique de 0 (grain de poussière) à 100 (Dieu)
    Voici les Caracteristiques:
    Force (STR): Mesure la puissance physique, la capacité à soulever des objets lourds et la force brute. (Exemple: attaques de mêlée, dégâts physiques, capacité à porter des charges, sauter, grimper.)
    Dextérité (DEX): Représente l’agilité, la rapidité et la précision des mouvements.(Exemple : initiative, attaques à distance, furtivité, esquive, etc)
    Constitution (CON): Reflète l’endurance physique et la robustesse du personnage. (Exemple : Points de vie (HP), résistance aux poisons et maladies, etc)
    Intelligence (INT): Mesure la logique, la mémoire et les connaissances académiques. (Exemple : compétences d'érudition, connaissance, etc)
    Sagesse (WIS): Indique la perception, l’intuition et la connexion au monde spirituel. (Exemple : perception passive, résistance aux illusions, etc)
    Charisme (CHA): Symbolise la force de personnalité, la persuasion et le charisme naturel. (Exemple : interactions sociales, charmes, etc.

    Ton résultat doit être sous la forme d'un dictionnaire Python valide sans nom de variable avant exactement comme cette exemple
    [{
        "ID" : 18940,
        "STR": 10,
        "DEX" : 22,
        "CON" : 45,
        "INT" : 63,
        "WIS" : 75,
        "CHA" : 90
        }
    }]
    """

    # number of character by prompt
    batch_size = 10

    # number of prompt to do
    num_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)

    print("num_batches =",num_batches)

    stat_list = []

    for i in range(num_batches):
        print("i =",i)

        chat = model.start_chat(history=[{'role': 'user', 'parts': [system_prompt]}])

        # create a df with 10 character
        batch = data.iloc[i * batch_size:(i + 1) * batch_size]

        response = chat.send_message(batch.to_json())

        clean_responce = response.text.replace("```", "").replace("python", "").replace("json", "")

        print(clean_responce)

        result_list = ast.literal_eval(clean_responce)

        stat_list += result_list

        time.sleep(1)
        
    stat_df = pd.DataFrame(stat_list)
    stat_df = stat_df[['ID','STR','DEX','CON','INT','WIS','CHA']]
    return stat_df

def merge_original_df_with_stat_df(original_df, stat_df):
    merge_df = pd.merge(stat_df,original_df, on='ID')
    return merge_df


character_df2 = pd.read_csv("characters_df2.csv")
character_df2 = character_df2[['ID','Name','Full Name','Aliases','Birth','Race','Gender','Powers','Publisher','Description']]

stat_df = generate_stats_with_gemini(character_df2)

character_df3 = merge_original_df_with_stat_df(character_df2, stat_df)

character_df3.to_csv("characters_df3.csv", index=False)